# Setup

In [1]:
import sys, os

# Move one level up from notebooks/ to project/
sys.path.append(os.path.abspath(".."))

# Question 1

Goal is to save the data from the API in a database. We start with the measurements. We want to have the following columns:

- measurementid (not in dataset by default)
- timestamp
- temperature
- groundtemperature
- feeltemperature
- windgusts
- windspeedBft
- humidity
- precipitation
- sunpower
- stationid

We create two module for the extraction and transformation of data:

- create extract.py
    - extract_json_to_df
- create transform.py
    - add_primary_key
    - select_columns

We then use the functions that we created to first extract the raw data, then to add a measurementid columns and then we select all the relevant columns.


In [2]:
from src import extract

# extract the data from the api
url = "https://data.buienradar.nl/2.0/feed/json"
df_raw = extract.extract_json_to_df(url, ["actual", "stationmeasurements"])

✅ Extracted 41 station measurements from Buienradar


Data is loaded, now we apply some transformations

In [3]:
from src import transform

# create the measurementid column
df_measurements = transform.add_primary_key(df_raw, "measurementid")

# the columns we want to select from the data
measurement_columns = [
    "measurementid",
    "timestamp",
    "temperature",
    "groundtemperature",
    "feeltemperature",
    "windgusts",
    "windspeedBft",
    "humidity",
    "precipitation",
    "sunpower",
    "stationid"
]

# select only the necessary columns from the data
df_measurements = transform.select_columns(df_measurements, measurement_columns)

Primary key column 'measurementid' added.
Selected 11 columns from the dataframe


# Question 2

In this step we want to extract all the unique stations from the measurements and save them in their own dataframe

Columsn we want to have in the dataframe:

- stationid
- stationname
- lat
- lon
- regio

We use the `select_columns` from the previous step to select the columns from the original stationmeasurements. And then we use a new function to find all the unique stations that have measurements in our dataset.


In [4]:
from src import transform

# the columns we want to select from the data
station_columns = [
    "stationid",
    "stationname",
    "lat",
    "lon",
    "regio"
]

df_stations = transform.select_columns(df_raw, station_columns)
df_stations = df_stations.drop_duplicates()

Selected 5 columns from the dataframe


# Question 3

Now we load both dataframes into a sqlite database. We need to create a new module for the loading of data. In this module we create functions to create a connection, to create a table and to create foreign key relation.

- create load.py
    - get_engine
    - load_df_to_sqlite
    - create_foreign_key
    

In [5]:
from src import load

load.load_df_to_sqlite(df_measurements, "measurements", "measurementid")
load.load_df_to_sqlite(df_stations, "stations", "stationid")

✅ Loaded 41 records into table 'measurements'
✅ Loaded 41 records into table 'stations'


Now we check if we can query the new tables.

In [13]:
%load_ext sql
%sql sqlite:///data/weather.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [14]:
%%sql
SELECT 
    *
FROM measurements m
INNER JOIN stations s 
    ON m.stationid = s.stationid
LIMIT 5;

 * sqlite:///data/weather.db
Done.


measurementid,timestamp,temperature,groundtemperature,feeltemperature,windgusts,windspeedBft,humidity,precipitation,sunpower,stationid,stationid_1,stationname,lat,lon,regio
1,2025-10-26T15:50:00,9.8,9.3,7.9,7.8,3.0,64.0,0.0,71.0,6391,6391,Meetstation Arcen,51.5,6.2,Venlo
2,2025-10-26T15:50:00,8.5,7.9,5.5,10.1,4.0,81.0,0.0,100.0,6275,6275,Meetstation Arnhem,52.07,5.88,Arnhem
3,2025-10-26T15:50:00,9.6,9.1,5.5,17.3,6.0,68.0,0.0,114.0,6249,6249,Meetstation Berkhout,52.65,4.98,Berkhout
4,2025-10-26T15:50:00,10.0,9.9,7.1,13.9,4.0,65.0,0.0,165.0,6260,6260,Meetstation De Bilt,52.1,5.18,Utrecht
5,2025-10-26T15:50:00,10.3,9.7,6.3,16.8,6.0,64.0,0.0,103.0,6235,6235,Meetstation Den Helder,52.92,4.78,Den Helder


It works!!

However, I have not formalized the foreign key relationship between the two tables

# Question 4

Here we show the ERD of the tables we just created

<img src="../images/ERD buienradar.drawio.png" width=600px height=200px />

# Question 5

We want to find out which weather station recorded the highest temperature. We can do this using a simple query where we 
- join the measurements on the stations
- filter out all the measurements where there was no temperature recorded
- order the rows based on the temperature (in descending order)
- select only the top row
- we are only interested in the stationid and stationname columns and the temperature that was measured

In [15]:
%%sql
SELECT 
    s.stationid,
    s.stationname,
    m.temperature
FROM measurements m
INNER JOIN stations s 
    ON m.stationid = s.stationid
WHERE m.temperature IS NOT NULL
ORDER BY m.temperature DESC
LIMIT 1;

 * sqlite:///data/weather.db
Done.


stationid,stationname,temperature
6277,Meetstation Lauwersoog,10.9


# Question 6

We now want to know the average temperature, since we cannot learn anything from the measurements that did not give a temperature we shall filter them out again.

The query below should return the average temperature for this day

In [16]:
%%sql
SELECT 
    AVG(m.temperature) as avg_temp
FROM measurements m
WHERE m.temperature IS NOT NULL

 * sqlite:///data/weather.db
Done.


avg_temp
9.524999999999999


# Question 7

We are looking for the maximum difference between the actual temperature and the feel temperature. To find this we will first need to create a CTE with a new column that find the absolute difference between the `temperature` column and the `feeltemperature` column. Then we will again find the highest value of this new column like in Question 5.

We will also need to make sure that both `temperature` and `feeltemperature` are not NULL

In [10]:
%%sql
WITH station_temp_diffs AS (
    SELECT 
        s.stationid,
        s.stationname,
        m.temperature,
        m.feeltemperature,
        ABS(m.temperature - m.feeltemperature) as temp_diff
    FROM measurements m
    INNER JOIN stations s 
        ON m.stationid = s.stationid
    WHERE m.temperature IS NOT NULL
        AND m.feeltemperature IS NOT NULL
)
SELECT 
    *
FROM station_temp_diffs
ORDER BY temp_diff DESC
LIMIT 1;

 * sqlite:///data/weather.db
Done.


stationid,stationname,temperature,feeltemperature,temp_diff
6242,Meetstation Vlieland,9.2,4.1,5.1


# Question 8

We are looking for which measuring station is in the North Sea.

We can use the regio column in the stations table. 
First we will look at which regions are in the stations table by selecting the unique values of the `regio` column.

In [11]:
%%sql
SELECT DISTINCT regio FROM stations 

 * sqlite:///data/weather.db
Done.


regio
Venlo
Arnhem
Berkhout
Utrecht
Den Helder
Eindhoven
Weert
Gilze Rijen
Goes
Oost-Overijssel


After inspection of the results we can see that there is a region called 'Noordzee' which is the North Sea. So we would want to select this station for our answer.

In [12]:
%%sql
SELECT 
    stationid,
    stationname
FROM stations
WHERE regio = 'Noordzee'

 * sqlite:///data/weather.db
Done.


stationid,stationname
6239,Meetstation Zeeplatform F-3


_note: if the dataset had been a lot larger with many more stations and regions this method of manually looking up the correct region name would have been infeasible. alternative methods would have been:_

- _finding the coordindate ranges of the north sea and querying the `lat` and `lon` fiels on that_
- _using textsearch on the `regio` column e.g. 
```WHERE lower(regio) like '%noord%' OR lower(regio) like '%north%'``` 
to reduce the possible regions to look through_

# Question 9A

We want to automate the process of fetching the weather data from the buienradar api to get all the measurements for a day.

A couple things would be nececssary for this:
- A scheduler that runs a script to fetch new data ([we can use cron for this](../scheduler/create_cron.sh))
- A system to run this on (we can create a VM in the cloud or I can use my own computer)
- We want to log the fetches
    - We can create a new table: fetch_logs
    - Our script needs to append a row to the table for each run
    - The fetch_logs table stores data about the fetch
        - status (was the fetch sucessful)
        - records (number of records that were fetched)
        - timestamp (timestamp of the fetch)
- our load processes should make sure to merge the new data with the existing data
    - For measurements
        - New combinations of (stationid, timestamp) are added as new rows
        - If a combination of (stationid, timestamp) already exists, we will overwrite this row with the new incoming data
    - For stations
        - If a new stationid is foubd in the latest fetch we must add it to the stations table.
 